# Week 5 - Pandas and Data Cleanup

#### Recap from Previous Class:
- Fetched data using Yelp API
- Analyzed and processed the data for the Top5 App
- Pulled duration data using Mapbox API
- Learned about the following concepts:
    - Pandas Data Structures
    - DataFrame and Series attributes
    - Selecting and Subsetting in DataFrames
    - Pass by Reference vs Pass by Values
    - Looping through DataFrame rows
    - Method Chaining
    - Apply method and lambda
    - Mapping values in DataFrames

#### Topics Covered in Videos:
- Missing Values
- Wrangling, Reshaping and Pivot Tables
- Data Analysis: Split-Apply-Combine and Pandas GroupBy
- Pandas str and regex

#### Our Focus for today:
1. Understand how to deal with missing values 
    - An in-depth review of recognizing patters and handling missing values in terms of the context
    - Use ML for imputation
2. Learn about dummy variables and their creation
3. Apply the concept of GroupBy-Split-Combine
4. Learn about Pivot, Reshape and Melt
5. Str and Regex in Python
    - Processing Text Data


<img src="./images/app_image.png" width="300">

#### <span style='color:blue'>Task 0: Fetching Yelp Data and Structuring DataFrame</span> 

In [1]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Extracting Yelp Key
from configparser import ConfigParser
config = ConfigParser()
config.read('config.ini')

API_KEY = config["YELP"]["api-key"]

In [3]:
# import data from yelp
import requests
import json

url = 'https://api.yelp.com/v3' + '/businesses/search'

url_params = {
    'location': 'Denver, CO',
    'limit': 50,
    'term': 'restaurant',
    'radius': 2000
}

headers = {'Authorization': API_KEY}

response = requests.get(url, headers=headers, params=url_params)
if response.status_code == 200:
    result = json.loads(response.content)

In [4]:
import pandas as pd
restaurant_df = pd.DataFrame(result['businesses'])

# Append data
for i in range(1, 5):
    url_params = {
        'location': 'Denver, CO',
        'limit': 50,
        'term': 'restaurant',
        'radius': 5000,
        'offset': (i*50)
    }
    if i == 4:
        url_params['limit'] = 40
    response = requests.get(url, headers=headers, params=url_params)
    try:
        result = json.loads(response.content)
        data = result['businesses']
        new_df = pd.DataFrame(data)
        restaurant_df = pd.concat([restaurant_df, new_df], ignore_index=True)
    except:
        print(response.content)


In [5]:
# create a deep copy
restaurant_df_clean = restaurant_df.copy(deep=True)

# capatialize columns
restaurant_df_clean.columns = restaurant_df_clean.columns.str.capitalize()

# Subset columns using loc
restaurant_df_clean = restaurant_df_clean.loc[:, ['Id', 'Name', 'Is_closed', 'Review_count', 'Categories', 
                                      'Rating', 'Coordinates', 'Transactions', 'Price', 'Location',
                                      'Distance', 'Business_hours']]

In [6]:
# extract display address from location
restaurant_df_clean['Location'] = restaurant_df_clean['Location'].apply(lambda x: x['display_address'])

In [7]:
# convert data types
restaurant_df_clean = restaurant_df_clean.astype({
    'Price': 'category',
    'Name': 'string',
    'Id':'string'
})

def alter_transactions(df):
    for transaction in ['delivery', 'pickup', 'restaurant_reservation']:
        df.loc[:, transaction] = 0
        df.loc[:, transaction] = df.loc[:, 'Transactions'].apply(lambda x: 1 if transaction in x else 0)

alter_transactions(restaurant_df_clean)

restaurant_df_clean.head(3)

,Id,Name,Is_closed,Review_count,Categories,Rating,Coordinates,Transactions,Price,Location,Distance,Business_hours,delivery,pickup,restaurant_reservation
0,uQd0TZKBprBXxF1aGSIAIA,Dew Drop Inn,False,65,"[{'alias': 'gastropubs', 'title': 'Gastropubs'...",4.6,"{'latitude': 39.7434, 'longitude': -104.9738192}",[],NaN,"[1033 E 17th Ave, Denver, CO 80218]",927.529237,"[{'open': [{'is_overnight': False, 'start': '1...",0,0,0
1,6F4_oM9oRzBDc7KeTEtvcw,Wild Corgi Pub,False,129,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.4,"{'latitude': 39.736985832484, 'longitude': -10...","[delivery, pickup]",NaN,"[1223 E 13th Ave, Denver, CO 80218]",231.913621,"[{'open': [{'is_overnight': False, 'start': '1...",1,1,0
2,qRLjMCH1ysrOl3ewkZ-2qQ,Pancho Poncho,False,22,"[{'alias': 'mexican', 'title': 'Mexican'}]",4.5,"{'latitude': 39.727133, 'longitude': -104.98216}",[],NaN,"[400 E 7th Ave, Denver, CO 80203]",1382.911408,"[{'open': [{'is_overnight': False, 'start': '1...",0,0,0


### Dealing with Missing Values

Understanding the nature of missing data is crucial for choosing appropriate handling methods, such as imputation techniques or adjustments in statistical models. The interpretation can significantly impact the validity of conclusions drawn from the data.

There are several ways to interpret missing values in data science, each with different implications for analysis and modeling. Here are some common interpretations:

1. Missing Completely at Random (MCAR):
   - Data is missing due to entirely random factors
   - No relationship between the missing data and other variables
   - Least problematic for analysis. Safe to remove MCAR data as it does not introduce bias into the analysis
   - E.g: Data missing due to technical issues or equipment malfunctions.

2. Missing at Random (MAR):
   - Missing data is related to other observed variables but not to the missing variable itself
   - Requires careful handling to avoid bias. Techniques like imputation or weighting can be used
   - E.g. Data missing due to known characteristics of the population, such as non-response in surveys.

3. Missing Not at Random (MNAR):
   - Missing data is related to the unobserved values themselves
   - Most challenging to handle and can lead to biased results
   - Requires specialized techniques like multiple imputation or selection models to avoid bias.
   - E.g. Data missing due to the nature of the variable (e.g., sensitive questions in surveys)

<img src="./images/missing_vals.png" width="400">

4. Structural missingness:
   - Data is missing due to the inherent structure of the data collection process
   - Example: Questions that are only asked if a previous question is answered in a certain way

5. Truncated data:
   - Values beyond a certain threshold are completely unobserved

6. Intermittent missingness:
   - Data is missing for some time points in a time series
   - Common in longitudinal studies

7. Dropout:
   - Subjects leave a study before its completion, leading to missing data in later time points

8. Informative missingness:
   - The fact that data is missing provides information about the underlying phenomenon

9. Non-response:
    - Subjects choose not to answer certain questions, which may indicate sensitivity or other factors


Additional Readings:
- https://www.geeksforgeeks.org/ml-handling-missing-values/
- https://www.freecodecamp.org/news/how-to-handle-missing-data-in-a-dataset/
- https://www.mastersindatascience.org/learning/how-to-deal-with-missing-data/
- https://www.linkedin.com/advice/3/what-some-common-causes-types-missing-values-datasets
- https://www.linkedin.com/advice/0/what-best-practices-identifying-handling-missing

#### <span style='color:blue'>Task 1: Handle Missing Price Values</span> 

In [10]:
# value count with missing values
restaurant_df_clean.loc[:, 'Price'].value_counts(dropna=False)

Price
$$      141
NaN      65
$$$      24
$         6
$$$$      4
Name: count, dtype: int64

In [18]:
restaurant_df_imputed = restaurant_df_clean.copy(deep=True)
# fill missing values with mode
restaurant_df_imputed.loc[:, 'Price'].fillna(restaurant_df_imputed.loc[:, 'Price'].mode()[0], inplace=False).value_counts(dropna=False)


Price
$$      206
$$$      24
$         6
$$$$      4
Name: count, dtype: int64

Let's try imputation with Random Forest.

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

# Prepare the data
X = restaurant_df_imputed.loc[:, ['Rating', 'Review_count', 'Price']]
y = restaurant_df_imputed['Price']

# Do label encoding
le = LabelEncoder()
y_train = le.fit_transform(y.dropna())

In [22]:
X_train = X.loc[X['Price'].notnull(), ['Rating', 'Review_count']]
X_test = X.loc[X['Price'].isnull(), ['Rating', 'Review_count']]

In [23]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [24]:
missing_price = restaurant_df_imputed['Price'].isnull()

# Impute missing values
restaurant_df_imputed.loc[missing_price, 'Price'] = \
le.inverse_transform(
    rf_model.predict(
        restaurant_df_imputed.loc[missing_price, ['Rating', 'Review_count']]
        )
)

In [25]:
# value counts
restaurant_df_imputed.loc[:, 'Price'].value_counts(dropna=False)

Price
$$      201
$$$      28
$         6
$$$$      5
Name: count, dtype: int64

#### Dummy Variables

- Representing categorical data: They allow us to include categorical variables in Machine Learning models that require numerical inputs.
- Avoiding ordinal relationships: Dummy coding prevents implying an ordinal relationship where none exists.
- Improved interpretability: Each dummy variable has a clear interpretation in the context of the model.

#### <span style='color:blue'>Task 2: Create dummy for categories</span> 

In [30]:
# check categories value
restaurant_df_imputed.loc[51, 'Categories']

['Indian', 'Vegan', 'Gluten-Free']

In [29]:
# in each row, create a category list
restaurant_df_imputed['Categories'] = restaurant_df_imputed['Categories'].apply(lambda x: [category['title'] for category in x])

In [31]:
# loop through categories using itertuples
category_set = dict()
for row in restaurant_df_imputed.itertuples():
    for category in row.Categories:
        category_set[category] = category_set.get(category, 0) + 1


In [33]:
# take top 10
category_set = dict(sorted(category_set.items(), key=lambda x: x[1], reverse=True)[:10])

In [34]:
# display category_set
category_set

{'Cocktail Bars': 45,
 'Bars': 40,
 'New American': 38,
 'Breakfast & Brunch': 34,
 'American': 26,
 'Mexican': 18,
 'Wine Bars': 17,
 'Italian': 17,
 'Sandwiches': 16,
 'Seafood': 14}

In [35]:
# create dummy variables
for category in category_set:
    restaurant_df_imputed[category] = restaurant_df_imputed['Categories'].apply(lambda x: 1 if category in x else 0)

In [36]:
restaurant_df_imputed.head(3)

,Id,Name,Is_closed,Review_count,Categories,Rating,Coordinates,Transactions,Price,Location,...,Cocktail Bars,Bars,New American,Breakfast & Brunch,American,Mexican,Wine Bars,Italian,Sandwiches,Seafood
0,uQd0TZKBprBXxF1aGSIAIA,Dew Drop Inn,False,65,"[Gastropubs, Cocktail Bars, Wine Bars]",4.6,"{'latitude': 39.7434, 'longitude': -104.9738192}",[],$$,"[1033 E 17th Ave, Denver, CO 80218]",...,1,0,0,0,0,0,1,0,0,0
1,6F4_oM9oRzBDc7KeTEtvcw,Wild Corgi Pub,False,129,"[Pubs, American]",4.4,"{'latitude': 39.736985832484, 'longitude': -10...","[delivery, pickup]",$$,"[1223 E 13th Ave, Denver, CO 80218]",...,0,0,0,0,1,0,0,0,0,0
2,qRLjMCH1ysrOl3ewkZ-2qQ,Pancho Poncho,False,22,[Mexican],4.5,"{'latitude': 39.727133, 'longitude': -104.98216}",[],$$,"[400 E 7th Ave, Denver, CO 80203]",...,0,0,0,0,0,1,0,0,0,0


We can also use pd.get_dummies for simpler ways of creating dummies for categorical variables.

In [37]:
# create dummy variables for Price column
pd.get_dummies(restaurant_df_imputed, columns=['Price']).head(3)

,Id,Name,Is_closed,Review_count,Categories,Rating,Coordinates,Transactions,Location,Distance,...,American,Mexican,Wine Bars,Italian,Sandwiches,Seafood,Price_$,Price_$$,Price_$$$,Price_$$$$
0,uQd0TZKBprBXxF1aGSIAIA,Dew Drop Inn,False,65,"[Gastropubs, Cocktail Bars, Wine Bars]",4.6,"{'latitude': 39.7434, 'longitude': -104.9738192}",[],"[1033 E 17th Ave, Denver, CO 80218]",927.529237,...,0,0,1,0,0,0,False,True,False,False
1,6F4_oM9oRzBDc7KeTEtvcw,Wild Corgi Pub,False,129,"[Pubs, American]",4.4,"{'latitude': 39.736985832484, 'longitude': -10...","[delivery, pickup]","[1223 E 13th Ave, Denver, CO 80218]",231.913621,...,1,0,0,0,0,0,False,True,False,False
2,qRLjMCH1ysrOl3ewkZ-2qQ,Pancho Poncho,False,22,[Mexican],4.5,"{'latitude': 39.727133, 'longitude': -104.98216}",[],"[400 E 7th Ave, Denver, CO 80203]",1382.911408,...,0,1,0,0,0,0,False,True,False,False


### Groupby-Split-Combine

- Group: Divide your data into groups based on some common characteristic.
- Split: Separate the data into these groups.
- Apply: Perform an operation or calculation on each group independently.

<img src="./images/groupby.png" width="600">


A list of all the functions can be found in the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#grouping-operations)


| Function          | Description                                                           |
|-------------------|-----------------------------------------------------------------------|
| **Aggregation Functions** |
| `sum()`           | Compute the sum of group values.                                      |
| `mean()`          | Compute the mean of group values.                                     |
| `median()`        | Compute the median of group values.                                   |
| `min()`           | Compute the minimum of group values.                                  |
| `max()`           | Compute the maximum of group values.                                  |
| `count()`         | Compute the count of group values.                                    |
| `size()`          | Compute the size of each group.                                       |
| `std()`           | Compute the standard deviation of group values.                       |
| `var()`           | Compute the variance of group values.                                 |
| `sem()`           | Compute the standard error of the mean of group values.               |
| `first()`         | Compute the first value of each group.                                |
| `last()`          | Compute the last value of each group.                                 |
| `nth(n)`          | Compute the nth value of each group.                                  |
| `prod()`          | Compute the product of group values.                                  |
| `any()`           | Returns True if any element in the group is True.                     |
| `all()`           | Returns True if all elements in the group are True.                   |
| **Transformation Functions** |
| `transform(func)` | Apply a function to each group and return a DataFrame with the same shape as the original. |
| **Apply Functions** |
| `apply(func)`     | Apply a function to each group and combine the results into a DataFrame, Series, or scalar depending on the function's output. |
| **Descriptive Statistics** |
| `describe()`      | Generate descriptive statistics for each group.                       |
| `quantile(q)`     | Compute the quantile of each group.                                   |
| `mad()`           | Compute the mean absolute deviation of group values.                  |
| `skew()`          | Compute the skewness of group values.                                 |
| `kurt()`          | Compute the kurtosis of group values.                                 |
| **Other Functions** |
| `agg(func)`       | Aggregate using one or more operations over the specified axis.       |
| `cumsum()`        | Compute the cumulative sum of group values.                           |
| `cumprod()`       | Compute the cumulative product of group values.                       |
| `cummin()`        | Compute the cumulative minimum of group values.                       |
| `cummax()`        | Compute the cumulative maximum of group values.                       |
| `head(n)`         | Return the first `n` rows of each group.                              |
| `tail(n)`         | Return the last `n` rows of each group.                               |
| `ngroup()`        | Number each group from 0 to the number of groups - 1.                 |
| `nth(n)`          | Take the nth row from each group.                                     |

#### <span style='color:blue'>Task 3: Get summarized information</span> 

In [44]:
restaurant_df_imputed.groupby('Price').count()

,Id,Name,Is_closed,Review_count,Categories,Rating,Coordinates,Transactions,Location,Distance,...,Cocktail Bars,Bars,New American,Breakfast & Brunch,American,Mexican,Wine Bars,Italian,Sandwiches,Seafood
Price,,,,,,,,,,,,,,,,,,,,,
$,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
$$,201,201,201,201,201,201,201,201,201,201,...,201,201,201,201,201,201,201,201,201,201
$$$,28,28,28,28,28,28,28,28,28,28,...,28,28,28,28,28,28,28,28,28,28
$$$$,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5


In [48]:
# Groupby price and count and average the ratings and review-count
restaurant_df_imputed.groupby('Price').agg({'Review_count': 'mean', 'Rating': 'mean'})

,Review_count,Rating
Price,,
$,313.000000,4.250000
$$,477.000000,4.284577
$$$,478.071429,4.232143
$$$$,194.600000,4.520000


In [53]:
# Groupby price and count for all restuarant categories
restaurant_df_imputed.groupby('Price')[list(category_set.keys())].sum()


,Cocktail Bars,Bars,New American,Breakfast & Brunch,American,Mexican,Wine Bars,Italian,Sandwiches,Seafood
Price,,,,,,,,,,
$,0,2,0,2,0,2,0,0,0,0
$$,38,34,24,30,24,16,14,12,15,10
$$$,5,4,11,2,2,0,1,5,1,4
$$$$,2,0,3,0,0,0,2,0,0,0


### Reshaping: Pivot and Melt

**Multi-indexing**: Multi-indexing in pandas allows you to work with more complex data structures by using multiple levels of indexes (row or column labels). This is particularly useful for handling data with multiple dimensions or hierarchical data.

In [54]:
data = {
    'Month': ['Jan', 'Jan', 'Jan', 'Jan', 'Feb', 'Feb', 'Feb', 'Feb'],
    'Region': ['North', 'North', 'South', 'South', 'North', 'North', 'South', 'South'],
    'Product': ['A', 'B', 'A', 'B', 'A', 'B', 'A', 'B'],
    'Sales': [100, 150, 200, 250, 110, 160, 210, 260]
}

df = pd.DataFrame(data)
df.set_index(['Month', 'Region'], inplace=True)
print(df)


             Product  Sales
Month Region               
Jan   North        A    100
      North        B    150
      South        A    200
      South        B    250
Feb   North        A    110
      North        B    160
      South        A    210
      South        B    260


**Pivot**: Pivoting is the process of transforming data from a long format to a wide format. In a wide format, we use unique values from a column to create new columns. In other words, spread rows into columns.

<img src="./images/pivot.png" width="600">

**Melt**: Melting is the opposite of pivoting. It transforms data from a wide format to a long format. In a long format, we have more rows and fewer columns. In other words, it gathers columns into rows.

<img src="./images/melt.png" width="600">

#### <span style='color:blue'>Task 4: Create top 5 table for each category</span> 

In [55]:
# Filter where Mexican is 1 and sortby rating
restaurant_df_imputed.loc[restaurant_df_imputed['Mexican'] == 1, ['Name', 'Review_count', 'Price', 'Rating']].sort_values('Rating', ascending=False).head(5)

,Name,Review_count,Price,Rating
98,House Of CoPa,4,$$,5.0
119,Pete’s Satire Lounge,8,$$,4.9
34,Pete’s Satire Lounge,8,$$,4.9
220,Mariscos El Charco,14,$$,4.9
209,Toro Food Concepts,27,$$,4.5


In [56]:
restaurant_df_imputed.columns


Index(['Id', 'Name', 'Is_closed', 'Review_count', 'Categories', 'Rating',
       'Coordinates', 'Transactions', 'Price', 'Location', 'Distance',
       'Business_hours', 'delivery', 'pickup', 'restaurant_reservation',
       'Cocktail Bars', 'Bars', 'New American', 'Breakfast & Brunch',
       'American', 'Mexican', 'Wine Bars', 'Italian', 'Sandwiches', 'Seafood'],
      dtype='object')

In [57]:
col_indices = [1, 5, 10] + list(range(15, 25))
restaurant_top5 = restaurant_df_imputed.copy(deep=True).iloc[:, col_indices]

In [60]:
restaurant_top5.head()

,Name,Rating,Distance,Cocktail Bars,Bars,New American,Breakfast & Brunch,American,Mexican,Wine Bars,Italian,Sandwiches,Seafood
0,Dew Drop Inn,4.6,927.529237,1,0,0,0,0,0,1,0,0,0
1,Wild Corgi Pub,4.4,231.913621,0,0,0,0,1,0,0,0,0,0
2,Pancho Poncho,4.5,1382.911408,0,0,0,0,0,1,0,0,0,0
3,Angelo’s Taverna,4.5,1351.392992,0,1,0,0,0,0,0,1,0,1
4,Sullivan Scrap Kitchen,4.5,901.864528,0,0,1,1,0,0,0,0,0,0


In [61]:
# Melt the DataFrame to have a single category column
melted = restaurant_top5.melt(
                id_vars=['Name', 'Rating', 'Distance'], 
                 value_vars= list(category_set.keys()),
                 var_name='Category', value_name='Is_Category')

In [69]:
melted.head()

,Name,Rating,Distance,Category,Is_Category
0,Dew Drop Inn,4.6,927.529237,Cocktail Bars,1
7,Revival Denver Public House,4.2,1124.024585,Cocktail Bars,1
8,The Mansion on Colfax,4.5,547.132348,Cocktail Bars,1
10,Reckless Noodle House,4.2,1414.966619,Cocktail Bars,1
12,sắp sửa,4.4,1256.797332,Cocktail Bars,1


In [67]:
# Filter only rows where Is_Category is 1
melted = melted[melted['Is_Category'] == 1]

In [70]:
# Remove the Is_Category column
melted = melted.drop(columns=['Is_Category'])

# Sort by Category, then Ratings, and Duration
melted = melted.sort_values(by=['Category', 'Rating', 'Distance'], ascending=[True, False, True])


In [74]:
melted.head(10)

,Name,Rating,Distance,Category
1124,LOB,5.0,2967.928157,American
1153,Lunchboxx,4.6,2921.008884,American
961,Wild Corgi Pub,4.4,231.913621,American
1099,Caldero,4.3,3056.661091,American
1003,Stoney's Uptown,4.2,930.504957,American
1103,Stoney's Uptown,4.2,930.504957,American
967,Revival Denver Public House,4.2,1124.024585,American
1127,5280 Burger Bar - Denver,4.1,1964.463941,American
1170,Sam's No 3 - Downtown,4.1,2487.819454,American
1190,Park Burger - RiNo,4.1,3009.947141,American


In [75]:

# Group by Category and take top 5
top_5 = melted.groupby('Category').head(5)


In [76]:
top_5

,Name,Rating,Distance,Category
1124,LOB,5.0,2967.928157,American
1153,Lunchboxx,4.6,2921.008884,American
961,Wild Corgi Pub,4.4,231.913621,American
1099,Caldero,4.3,3056.661091,American
1003,Stoney's Uptown,4.2,930.504957,American
455,Fireside The Bar,4.8,1984.323265,Bars
251,Pho & Bar,4.6,854.545539,Bars
390,McDevitt Taco Supply,4.6,3991.967322,Bars
243,Angelo’s Taverna,4.5,1351.392992,Bars
311,Cart-Driver,4.5,2802.200029,Bars


In [78]:

# Create a ranking within each category
top_5['Rank'] = top_5.groupby('Category').cumcount() + 1
top_5


,Name,Rating,Distance,Category,Rank
1124,LOB,5.0,2967.928157,American,1
1153,Lunchboxx,4.6,2921.008884,American,2
961,Wild Corgi Pub,4.4,231.913621,American,3
1099,Caldero,4.3,3056.661091,American,4
1003,Stoney's Uptown,4.2,930.504957,American,5
455,Fireside The Bar,4.8,1984.323265,Bars,1
251,Pho & Bar,4.6,854.545539,Bars,2
390,McDevitt Taco Supply,4.6,3991.967322,Bars,3
243,Angelo’s Taverna,4.5,1351.392992,Bars,4
311,Cart-Driver,4.5,2802.200029,Bars,5


In [79]:

# Set Multi-Index
top_5.set_index(['Category', 'Rank'], inplace=True)


In [80]:
top_5

Name  Rating  \
Category           Rank                                                  
American           1                                       LOB     5.0   
                   2                                 Lunchboxx     4.6   
                   3                            Wild Corgi Pub     4.4   
                   4                                   Caldero     4.3   
                   5                           Stoney's Uptown     4.2   
Bars               1                          Fireside The Bar     4.8   
                   2                                 Pho & Bar     4.6   
                   3                      McDevitt Taco Supply     4.6   
                   4                          Angelo’s Taverna     4.5   
                   5                               Cart-Driver     4.5   
Breakfast & Brunch 1                               Cafe Miriam     4.8   
                   2                             Taco Uprising     4.7   
                   3                                Leven Deli     4.6   
                   4                               Welton Room     4.6   
                   5                    Sullivan Scrap Kitchen     4.5   
Cocktail Bars      1                        Mariscos El Charco     4.9   
                   2                             Ukiyo Omakase     4.8   
                   3                                    Salita     4.7   
                   4                             Taco Uprising     4.7   
                   5                                     The W     4.7   
Italian            1     Vinny and Marie’s Italian Street Food     5.0   
                   2                          Angelo’s Taverna     4.5   
                   3                               Cart-Driver     4.5   
                   4                                DiFranco's     4.4   
                   5                               The Plimoth     4.4   
Mexican            1                             House Of CoPa     5.0   
                   2                      Pete’s Satire Lounge     4.9   
                   3                      Pete’s Satire Lounge     4.9   
                   4                        Mariscos El Charco     4.9   
                   5                        Toro Food Concepts     4.5   
New American       1                             Margot Denver     5.0   
                   2                           30Forth Kitchen     5.0   
                   3             Solutions Lounge & Restaurant     4.7   
                   4                                   Pony Up     4.6   
                   5                                  Nocturne     4.6   
Sandwiches         1                       The Porchetta House     4.8   
                   2             Carne - Argentina Street Food     4.8   
                   3                      Gaia Masala & Burger     4.6   
                   4                      Gaia Masala & Burger     4.6   
                   5                                Leven Deli     4.6   
Seafood            1                        Mariscos El Charco     4.9   
                   2                Crab & Shell Cajun Seafood     4.6   
                   3                Crab & Shell Cajun Seafood     4.6   
                   4                               Fish N Beer     4.6   
                   5                          Angelo’s Taverna     4.5   
Wine Bars          1                             Margot Denver     5.0   
                   2                                     The W     4.7   
                   3                              Dew Drop Inn     4.6   
                   4                                Leven Deli     4.6   
                   5                                    Beckon     4.6   

                            Distance  
Category           Rank               
American           1     2967.928157  
                   2     2921.008884  
                   3      231.913621  
              

### In Class Exercise: https://github.com/rahimrasool/tools1_week5/blob/main/In_class_exercise.ipynb

### Str and Regex

Some of the most essential string methods in Pandas:

1. **`str.lower()`**: Converts strings in the Series to lowercase.
2. **`str.upper()`**: Converts strings in the Series to uppercase.
3. **`str.title()`**: Converts strings in the Series to titlecase (each word starts with an uppercase letter).
4. **`str.capitalize()`**: Capitalizes the first character of each string in the Series.
5. **`str.strip()`**: Removes leading and trailing whitespace from each string in the Series.
6. **`str.lstrip()`**: Removes leading whitespace from each string in the Series.
7. **`str.rstrip()`**: Removes trailing whitespace from each string in the Series.
8. **`str.replace(old, new)`**: Replaces occurrences of a substring within each string with another substring.
9. **`str.cat(sep='')`**: Concatenates strings in the Series with a specified separator.
10. **`str.contains(pattern)`**: Checks if strings in the Series contain a pattern.
11. **`str.startswith(pattern)`**: Checks if strings in the Series start with a pattern.
12. **`str.endswith(pattern)`**: Checks if strings in the Series end with a pattern.
13. **`str.find(sub)`**: Finds the lowest index of the substring.
14. **`str.len()`**: Computes the length of each string in the Series.
15. **`str.split(pat=None, n=-1, expand=False)`**: Splits strings in the Series by a specified delimiter.
16. **`str.join(sep)`**: Joins lists in the Series into a single string with a specified separator.



In [81]:
restaurant_df_imputed.Location = restaurant_df_imputed.Location.apply(lambda x: ",".join(x))
restaurant_df_imputed.Location.head(2)

0    1033 E 17th Ave,Denver, CO 80218
1    1223 E 13th Ave,Denver, CO 80218
Name: Location, dtype: object

In [96]:
'1223 E 13thCO 80218 CO'.split()[-1]

'CO'

In [87]:
restaurant_df_imputed['Zipcode'] = restaurant_df_imputed.Location.apply(lambda x: x.split()[-1])


In [88]:
restaurant_df_imputed['Zipcode']

0      80218
1      80218
2      80203
3      80203
4      80218
       ...  
235    80218
236    80209
237    80218
238    80211
239    80203
Name: Zipcode, Length: 240, dtype: object

In [97]:
import re
re.search(r"\b\d{5}\b", '1033 E 17th Ave 80218 CO').group(0)

'80218'

### DateTime in Pandas

Temporal data (consisting of date and time stamps) is highly common in data analytics and can be processed to create useful features. Pandas offers the ability to work with time series information in various formats. This can enable us to perform several operations on datetime values such as sorting, predicting or categorizing data by certain time periods.

A convenient format for date and time data that Pandas uses is datetime from Python’s datetime library. This provides the ability for manipulating dates and times. We will work with datetime.datetime type from this library, which is a combination of both date and time. Pandas will assign either datetime64[ns] or datetime64[ns, tz] data type to such a format.

Series and DataFrame have extended data type support and functionality for datetime. In order to create this format from a Series or list consisting of timestamps in String or Object form (as is in the case of stops_df dataframe), we will use Pandas’ .to_datetime() function. A single scalar value will be converted to Timestamp data type, whereas, in a Series, the Pandas object data type will convert to datetime64[ns].

In [98]:
# converting a single scalar value
pd.to_datetime("2010/11/12")

Timestamp('2010-11-12 00:00:00')

In [99]:
import numpy as np
dates = pd.date_range(start='2024-01-01', end='2024-01-31')
stores = ['Littleton', 'Aurora', 'Highlands']
products = ['Airmax', 'Airforce', 'Jordans']
sales_data = pd.DataFrame({
    'date': np.random.choice(dates, size=300),
    'store': np.random.choice(stores, size=300),
    'product': np.random.choice(products, size=300),
    'sales': np.random.randint(10, 100, size=300)
})

In [100]:
sales_data.sample(20)

,date,store,product,sales
175,2024-01-31,Aurora,Airforce,66
278,2024-01-12,Highlands,Airmax,20
148,2024-01-02,Highlands,Airforce,23
181,2024-01-22,Littleton,Airforce,99
180,2024-01-23,Littleton,Jordans,48
31,2024-01-14,Littleton,Airmax,45
233,2024-01-15,Aurora,Jordans,95
82,2024-01-04,Littleton,Airmax,47
67,2024-01-16,Highlands,Airmax,32
117,2024-01-11,Aurora,Airforce,33


Every Timestamp has a set of time/date properties or temporal features that can be extracted.

For a Series of type datetime, we can use the .dt accessor to extract these properties. This [table](https://pandas.pydata.org/docs/user_guide/timeseries.html#time-date-components) lists all the properties that can be accessed. We will use it to get the day of the week (to classify if the sale was on a weekend or a weekday). We will assign it to a new column in the DataFrame.

In [101]:
sales_data.loc[:,'Weekday'] = sales_data.loc[:,'date'].dt.weekday
sales_data.sample(10)

,date,store,product,sales,Weekday
278,2024-01-12,Highlands,Airmax,20,4
297,2024-01-11,Aurora,Airforce,98,3
262,2024-01-31,Aurora,Airforce,30,2
193,2024-01-10,Aurora,Airforce,46,2
142,2024-01-18,Aurora,Airmax,86,3
265,2024-01-31,Aurora,Jordans,58,2
21,2024-01-15,Highlands,Airmax,50,0
214,2024-01-09,Aurora,Jordans,21,1
109,2024-01-29,Littleton,Airforce,27,0
161,2024-01-28,Littleton,Airmax,32,6


### Dask and RAPIDS